<a href="https://colab.research.google.com/github/pranaya-mathur/Deep-Learning-Projects/blob/master/Document_Search_Own_Data_7th_Sept_23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab,preprocessing,elasticsearch,inference]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
  Obtaining dependency information for farm-haystack[colab,elasticsearch,inference,preprocessing] from https://files.pythonhosted.org/packages/a3/e3/8a7ebb563df04300c9e2387ac31e8ffda809b21f276faa838b32cf4a74a7/farm_haystack-1.20.0-py3-none-any.whl.metadata
  Obtaining dependency information for canals==0.7.0 from https://files.pythonhosted.org/packages/08/2f/62e9e455c4cf183ebf6672c47befabb6a36a37a8cdc86256a5632d6ace10/canals-0.7.0-py3-none-any.whl.metadata
  Obtaining dependency information for events from https://files.pythonhosted.org/packages/25/ed/e47dec0626edd468c84c04d97769e7ab4ea6457b7f54dcb3f72b17fcd876/Events-0.5-py3-none-any.whl.metadata
  Obtaining dependency information for posthog from https://files.pythonhosted.org/packages/a7/73/35758818228c70348be4c3c6

In [2]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2

In [3]:
%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch

In [4]:
import time

time.sleep(30)

In [5]:
import os
from haystack.document_stores import ElasticsearchDocumentStore

# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")

document_store = ElasticsearchDocumentStore(host=host, username="", password="", index='aurelius')

In [6]:
import requests
data = requests.get('https://raw.githubusercontent.com/jamescalam/transformers/main/data/text/meditations/clean.txt')
text = data.text.split('\n')

In [7]:
text[:3]

['From my grandfather Verus I learned good morals and the government of my temper.',
 'From the reputation and remembrance of my father, modesty and a manly character.',
 'From my mother, piety and beneficence, and abstinence, not only from evil deeds, but even from evil thoughts; and further, simplicity in my way of living, far removed from the habits of the rich.']

In [8]:
requests.get('http://localhost:9200/_cluster/health').json()

{'cluster_name': 'elasticsearch',
 'status': 'yellow',
 'timed_out': False,
 'number_of_nodes': 1,
 'number_of_data_nodes': 1,
 'active_primary_shards': 2,
 'active_shards': 2,
 'relocating_shards': 0,
 'initializing_shards': 0,
 'unassigned_shards': 2,
 'delayed_unassigned_shards': 0,
 'number_of_pending_tasks': 0,
 'number_of_in_flight_fetch': 0,
 'task_max_waiting_in_queue_millis': 0,
 'active_shards_percent_as_number': 50.0}

In [9]:
print(requests.get('http://localhost:9200/_cat/indices').text)

yellow open aurelius B9T4fiB0Rti3rGeLDQoCsw 1 1 0 0 208b 208b
yellow open label    -6p5Ycv7Sb6TJS5OwS75Ig 1 1 0 0 208b 208b



In [10]:
data_json = [
    {
        'content': paragraph,
        'meta': {
            'source': 'meditations'
        }
    } for paragraph in text
]

In [11]:
data_json[:3]

[{'content': 'From my grandfather Verus I learned good morals and the government of my temper.',
  'meta': {'source': 'meditations'}},
 {'content': 'From the reputation and remembrance of my father, modesty and a manly character.',
  'meta': {'source': 'meditations'}},
 {'content': 'From my mother, piety and beneficence, and abstinence, not only from evil deeds, but even from evil thoughts; and further, simplicity in my way of living, far removed from the habits of the rich.',
  'meta': {'source': 'meditations'}}]

In [12]:
len(data_json)

507

In [14]:
document_store.write_documents(data_json)

In [15]:
requests.get('http://localhost:9200/aurelius/_count').json()

{'count': 507,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}

In [17]:
from haystack.nodes import BM25Retriever
# from haystack.reader.farm import FARMReader

# retriever = BM25Retriever(document_store)  # BM25
# reader = FARMReader(model_name_or_path='deepset/bert-base-cased-squad2',
#                     context_window_size=1500,
#                     use_gpu=True)

In [19]:
!pip install git+https://github.com/deepset-ai/haystack.git

  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-req-build-r_4lz3wd
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-req-build-r_4lz3wd
  Resolved https://github.com/deepset-ai/haystack.git to commit b7bea3ae9c59a5e2bd2228a355bcc562b3f80ba6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Obtaining dependency information for canals==0.8.0 from https://files.pythonhosted.org/packages/92/63/d0f20dd35a3f770baf4359f79635d7fb723ea9bf88800644fbf260c0bd4a/canals-0.8.0-py3-none-any.whl.metadata
  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/ae/59/911d6e5f1d7514d79c527067643376cddcf4cb8d1728e599b3b03ab51c69/openai-0.28.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.9/792.9 kB 9.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to 

In [21]:
from haystack.nodes import FARMReader

retriever = BM25Retriever(document_store)  # BM25
reader = FARMReader(model_name_or_path='deepset/bert-base-cased-squad2',
                    context_window_size=1500,
                    use_gpu=True)

Some weights of the model checkpoint at deepset/bert-base-cased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [23]:
from haystack.pipelines import ExtractiveQAPipeline

qa = ExtractiveQAPipeline(reader=reader, retriever=retriever)

In [24]:
qa.run(query='What did your grandfather teach?')

Inferencing Samples: 100%|██████████| 1/1 [00:03<00:00,  3.62s/ Batches]


{'query': 'What did your grandfather teach?',
 'no_ans_gap': -3.121262550354004,
 'answers': [<Answer {'answer': 'good morals and the government of my temper', 'type': 'extractive', 'score': 0.16657456755638123, 'context': 'From my grandfather Verus I learned good morals and the government of my temper.', 'offsets_in_document': [{'start': 36, 'end': 79}], 'offsets_in_context': [{'start': 36, 'end': 79}], 'document_ids': ['ecc466cb76ed8dedab8752cc84e411b4'], 'meta': {'source': 'meditations'}}>,
  <Answer {'answer': 'thy life under thy mother', 'type': 'extractive', 'score': 0.024128902703523636, 'context': 'Termination of activity, cessation from movement and opinion, and in a sense their death, is no evil. Turn thy thoughts now to the consideration of thy life, thy life as a child, as a youth, thy manhood, thy old age, for in these also every change was a death. Is this anything to fear? Turn thy thoughts now to thy life under thy grandfather, then to thy life under thy mother, then to